In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg19 import VGG19,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [4]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [5]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.2,
                                         rotation_range=15,
                                         height_shift_range=0.2,
                                         width_shift_range=0.2,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [6]:
vgg19_base_model = VGG19(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg19_base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
# x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='softmax')(x)

vgg19_model = Model(inputs=vgg19_base_model.input, outputs=x)

In [7]:
for layer in vgg19_model.layers[:22]:
    layer.trainable = False

for i,layer in enumerate(vgg19_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 False
13 block4_conv2 False
14 block4_conv3 False
15 block4_conv4 False
16 block4_pool False
17 block5_conv1 False
18 block5_conv2 False
19 block5_conv3 False
20 block5_conv4 False
21 block5_pool False
22 flatten_1 True
23 dense_1 True
24 batch_normalization_1 True
25 dropout_1 True
26 dense_2 True
27 batch_normalization_2 True
28 dropout_2 True
29 dense_3 True


In [8]:
vgg19_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=1e-3),
                    metrics=['accuracy'])

# fine-tune the model
vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

vgg19_model.save('vgg19_model_fc.h5')

Epoch 1/15
156/156 [==============================] - 364s 2s/step - loss: 1.9199 - acc: 0.4101 - val_loss: 0.8077 - val_acc: 0.7587
Epoch 2/15
156/156 [==============================] - 365s 2s/step - loss: 0.7998 - acc: 0.7298 - val_loss: 0.7830 - val_acc: 0.7483
Epoch 3/15
156/156 [==============================] - 356s 2s/step - loss: 0.5281 - acc: 0.8238 - val_loss: 1.0336 - val_acc: 0.7491
Epoch 4/15
156/156 [==============================] - 360s 2s/step - loss: 0.4223 - acc: 0.8626 - val_loss: 0.9641 - val_acc: 0.7374
Epoch 5/15
156/156 [==============================] - 352s 2s/step - loss: 0.3727 - acc: 0.8786 - val_loss: 0.7610 - val_acc: 0.7744
Epoch 6/15
156/156 [==============================] - 355s 2s/step - loss: 0.3246 - acc: 0.8914 - val_loss: 1.2188 - val_acc: 0.6964
Epoch 7/15
156/156 [==============================] - 352s 2s/step - loss: 0.2987 - acc: 0.9033 - val_loss: 1.3576 - val_acc: 0.6877
Epoch 8/15
156/156 [==============================] - 353s 2s/step - 

In [9]:
vgg19_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=5e-4),
#                     optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

Epoch 1/10
156/156 [==============================] - 362s 2s/step - loss: 0.2479 - acc: 0.9216 - val_loss: 0.9352 - val_acc: 0.7422
Epoch 2/10
156/156 [==============================] - 357s 2s/step - loss: 0.2234 - acc: 0.9276 - val_loss: 1.0709 - val_acc: 0.7326
Epoch 3/10
156/156 [==============================] - 359s 2s/step - loss: 0.2188 - acc: 0.9319 - val_loss: 1.2819 - val_acc: 0.7156
Epoch 4/10
156/156 [==============================] - 359s 2s/step - loss: 0.2128 - acc: 0.9318 - val_loss: 0.9250 - val_acc: 0.7609
Epoch 5/10
156/156 [==============================] - 362s 2s/step - loss: 0.1984 - acc: 0.9361 - val_loss: 1.1876 - val_acc: 0.7156
Epoch 6/10
156/156 [==============================] - 355s 2s/step - loss: 0.1819 - acc: 0.9407 - val_loss: 1.2859 - val_acc: 0.6747
Epoch 7/10
156/156 [==============================] - 355s 2s/step - loss: 0.1811 - acc: 0.9402 - val_loss: 1.1803 - val_acc: 0.7099


In [10]:
vgg19_model.save('vgg19_model_fc.h5')

In [13]:
vgg19_model = load_model('vgg19_model_fc.h5')

for layer in vgg19_model.layers[:17]:
    layer.trainable = False
    
for layer in vgg19_model.layers[17:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), #])
                                   EarlyStopping(patience=3)])

vgg19_model.save('vgg19_model_17.h5')

Epoch 1/20
156/156 [==============================] - 360s 2s/step - loss: 0.1891 - acc: 0.9400 - val_loss: 1.4136 - val_acc: 0.6272
Epoch 2/20
156/156 [==============================] - 352s 2s/step - loss: 0.0903 - acc: 0.9742 - val_loss: 0.8841 - val_acc: 0.8515
Epoch 3/20
156/156 [==============================] - 359s 2s/step - loss: 0.0558 - acc: 0.9836 - val_loss: 0.9470 - val_acc: 0.8389
Epoch 4/20
156/156 [==============================] - 357s 2s/step - loss: 0.0394 - acc: 0.9884 - val_loss: 0.6840 - val_acc: 0.8402
Epoch 5/20
156/156 [==============================] - 354s 2s/step - loss: 0.0348 - acc: 0.9900 - val_loss: 0.6166 - val_acc: 0.8558
Epoch 6/20
156/156 [==============================] - 358s 2s/step - loss: 0.0246 - acc: 0.9934 - val_loss: 1.0433 - val_acc: 0.7334
Epoch 7/20
156/156 [==============================] - 360s 2s/step - loss: 0.0277 - acc: 0.9919 - val_loss: 0.8055 - val_acc: 0.8179
Epoch 8/20
156/156 [==============================] - 362s 2s/step - 

In [14]:
vgg19_model = load_model('vgg19_model_17.h5')

for layer in vgg19_model.layers[:12]:
    layer.trainable = False
    
for layer in vgg19_model.layers[12:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), #])
                                   EarlyStopping(patience=2)])

vgg19_model.save('vgg19_model_12.h5')

Epoch 1/20
156/156 [==============================] - 377s 2s/step - loss: 0.0783 - acc: 0.9741 - val_loss: 1.0680 - val_acc: 0.8255
Epoch 2/20
156/156 [==============================] - 371s 2s/step - loss: 0.0517 - acc: 0.9840 - val_loss: 0.5333 - val_acc: 0.9164
Epoch 3/20
156/156 [==============================] - 371s 2s/step - loss: 0.0307 - acc: 0.9916 - val_loss: 1.1854 - val_acc: 0.8310
Epoch 4/20
156/156 [==============================] - 372s 2s/step - loss: 0.0397 - acc: 0.9885 - val_loss: 0.7603 - val_acc: 0.8663


In [15]:
vgg19_model = load_model('vgg19_model_12.h5')

for layer in vgg19_model.layers[:7]:
    layer.trainable = False
    
for layer in vgg19_model.layers[7:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
                     optimizer=optimizers.Adam(lr=5e-5),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'),
                                    EarlyStopping(patience=3)])

vgg19_model.save('vgg19_model_7.h5')

Epoch 1/20
156/156 [==============================] - 501s 3s/step - loss: 0.0244 - acc: 0.9931 - val_loss: 0.5036 - val_acc: 0.8971
Epoch 2/20
156/156 [==============================] - 494s 3s/step - loss: 0.0234 - acc: 0.9931 - val_loss: 0.4291 - val_acc: 0.8876
Epoch 3/20
156/156 [==============================] - 494s 3s/step - loss: 0.0221 - acc: 0.9937 - val_loss: 0.4307 - val_acc: 0.9085
Epoch 4/20
156/156 [==============================] - 494s 3s/step - loss: 0.0138 - acc: 0.9962 - val_loss: 0.6014 - val_acc: 0.8863
Epoch 5/20
156/156 [==============================] - 493s 3s/step - loss: 0.0151 - acc: 0.9957 - val_loss: 0.7536 - val_acc: 0.8741


In [6]:
# vgg19_model = load_model('vgg19_model_7.h5')

    
# vgg19_model.compile(loss='categorical_crossentropy',
#                      optimizer=optimizers.Adam(lr=1e-5),
#                     metrics=['accuracy'])  

# vgg19_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=20,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size,
#                         callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
#                                    CSVLogger('vgg19_fine_tune'),
#                                     EarlyStopping(patience=3)])


In [7]:
vgg19_model = load_model('vgg19_model_7.h5')

for layer in vgg19_model.layers[:4]:
    layer.trainable = False
    
for layer in vgg19_model.layers[4:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
                     optimizer=optimizers.Adam(lr=5e-5),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'),
                                    EarlyStopping(patience=3)])

vgg19_model.save('vgg19_model_4.h5')

Epoch 1/20
156/156 [==============================] - 494s 3s/step - loss: 0.0209 - acc: 0.9939 - val_loss: 0.3549 - val_acc: 0.9236
Epoch 2/20
156/156 [==============================] - 325s 2s/step - loss: 0.0122 - acc: 0.9964 - val_loss: 0.8674 - val_acc: 0.8105
Epoch 3/20
156/156 [==============================] - 322s 2s/step - loss: 0.0121 - acc: 0.9966 - val_loss: 0.5248 - val_acc: 0.8942
Epoch 4/20
156/156 [==============================] - 321s 2s/step - loss: 0.0125 - acc: 0.9966 - val_loss: 0.7803 - val_acc: 0.8519


In [8]:
vgg19_model = load_model('vgg19_model_4.h5')

for layer in vgg19_model.layers:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
                     optimizer=optimizers.Adam(lr=1e-5),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'),
                                    EarlyStopping(patience=3)])

#vgg19_model.save('vgg19_model_all.h5')

Epoch 1/20
156/156 [==============================] - 320s 2s/step - loss: 0.0048 - acc: 0.9988 - val_loss: 0.4295 - val_acc: 0.9219
Epoch 2/20
156/156 [==============================] - 326s 2s/step - loss: 0.0034 - acc: 0.9993 - val_loss: 0.4328 - val_acc: 0.9177
Epoch 3/20
156/156 [==============================] - 325s 2s/step - loss: 0.0024 - acc: 0.9995 - val_loss: 0.5127 - val_acc: 0.9081
Epoch 4/20
156/156 [==============================] - 322s 2s/step - loss: 0.0017 - acc: 0.9997 - val_loss: 0.5438 - val_acc: 0.9024


In [9]:
vgg19_model.save('vgg19_model_all.h5')

In [10]:
vgg19_model.compile(loss='categorical_crossentropy',
                     optimizer=optimizers.Adam(lr=5e-6),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'),
                                    EarlyStopping(patience=5)])

Epoch 1/20
156/156 [==============================] - 321s 2s/step - loss: 0.0018 - acc: 0.9996 - val_loss: 0.3753 - val_acc: 0.9262
Epoch 2/20
156/156 [==============================] - 326s 2s/step - loss: 0.0011 - acc: 0.9998 - val_loss: 0.3590 - val_acc: 0.9290
Epoch 3/20
156/156 [==============================] - 324s 2s/step - loss: 8.0702e-04 - acc: 0.9999 - val_loss: 0.4528 - val_acc: 0.9142
Epoch 4/20
156/156 [==============================] - 321s 2s/step - loss: 0.0010 - acc: 0.9999 - val_loss: 0.5394 - val_acc: 0.8920
Epoch 5/20
156/156 [==============================] - 321s 2s/step - loss: 0.0010 - acc: 0.9998 - val_loss: 0.4811 - val_acc: 0.9011
Epoch 6/20
156/156 [==============================] - 322s 2s/step - loss: 0.0012 - acc: 0.9997 - val_loss: 0.4944 - val_acc: 0.8959
Epoch 7/20
156/156 [==============================] - 316s 2s/step - loss: 8.2019e-04 - acc: 0.9999 - val_loss: 0.5197 - val_acc: 0.8937


In [11]:
vgg19_model.save('vgg19_final_model.h5')

In [12]:
vgg19_model = load_model('vgg19_final_model.h5')

test_model(vgg19_model,test_path,csv='vgg19_submission_0430.csv')